<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/CaptumLongformerSequenceClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import dependencies

In [2]:
import sys
sys.path.append('/content/drive/My Drive/{}'.format("cogs402longformer/"))

In [3]:
pip install transformers --quiet

     |████████████████████████████████| 4.4 MB 17.4 MB/s 
     |████████████████████████████████| 101 kB 13.2 MB/s 
     |████████████████████████████████| 6.6 MB 75.3 MB/s 
     |████████████████████████████████| 596 kB 89.9 MB/s 


In [4]:
pip install captum --quiet

     |████████████████████████████████| 1.4 MB 15.3 MB/s 


In [2]:
pip install datasets --quiet

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [4]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import pandas as pd

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Import model

In [6]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'danielhou13/longformer-finetuned_papers_v2'
#model_path = 'danielhou13/longformer-finetuned-new-cogs402'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Create functions that give us the input ids and the position ids for the text we want to examine

In [7]:
def predict(inputs, position_ids=None):
    output = model(inputs,
                   position_ids=position_ids)
    return output.logits

In [16]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [8]:
max_length = 2048
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids


Import dataset and take one example from it for testing purposes

In [9]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["test"]

Using custom data configuration danielhou13--cogs402dataset-144b958ac1a53abb
Reusing dataset parquet (/root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-144b958ac1a53abb/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# cogs402_ds2 = load_dataset('hyperpartisan_news_detection', 'bypublisher')['validation']
# val_size = 5000
# val_indices = np.random.randint(0, len(cogs402_ds2), val_size)
# val_ds = cogs402_ds2.select(val_indices)
# labels2 = map(int, val_ds['hyperpartisan'])
# labels2 = list(labels2)
# val_ds = val_ds.add_column("labels", labels2)

In [22]:
testval = 976
text = cogs402_ds['text'][testval]
label = cogs402_ds['labels'][testval]
print(label)

1


In [ ]:
# testval = 923
# text = cogs402_ds2['text'][testval]
# label = cogs402_ds2['labels'][testval]
# print(label)

In [10]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None):
    preds = predict(inputs,
                   position_ids=position_ids
                   )
    return torch.softmax(preds, dim = 1)

Perform Layer Integrated Gradients using the longformer's embeddings

In [11]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

In [23]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
# attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [24]:
attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    target=label,
                                    n_steps=250,
                                    internal_batch_size = 2)

In [12]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.linalg.norm(attributions)
    return attributions

In [25]:
attributions_sum = summarize_attributions(attributions)

In [26]:
score = predict(input_ids)

Visualize the attributions for the words

In [27]:
# storing couple samples in an array for visualization purposes
score_vis = viz.VisualizationDataRecord(
                        attributions_sum,
                        torch.softmax(score, dim = 1).max(),
                        torch.argmax(torch.softmax(score, dim = 1)),
                        label,
                        text,
                        attributions_sum.sum(),       
                        all_tokens,
                        delta)

print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


See which words had the strongest (most positive and most negative) attributions

In [13]:
def get_topk_attributed_tokens(attrs, k=20):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [14]:
def get_botk_attributed_tokens(attrs, k=20):
    values, indices = torch.topk(attrs, k, largest=False)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [28]:
top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_sum)
bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_sum)

df_high = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(top_words_start, top_word_ind_start, top_words_val_start)]})

df_low = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(bot_words_start, bot_word_ind_start, bot_words_val_start)]})
# df_start.style.apply(['cell_ids: False'])

# ['{}({})'.format(token, str(i)) for i, token in enumerate(all_tokens)]

In [29]:
df_high

,"Word(Index), Attribution"
0,". (1307), 0.22"
1,"Ġmatch (279), 0.21"
2,". (1688), 0.21"
3,"Ġwe (313), 0.17"
4,". (1656), 0.13"
5,". (669), 0.13"
6,"Ġthe (280), 0.13"
7,". (214), 0.12"
8,". (1072), 0.11"
9,"Ġthe (1171), 0.11"


In [30]:
df_low

,"Word(Index), Attribution"
0,". (251), -0.29"
1,". (1295), -0.27"
2,". (501), -0.19"
3,". (1195), -0.18"
4,". (1044), -0.17"
5,". (1584), -0.16"
6,". (151), -0.14"
7,"Ġthe (222), -0.14"
8,". (287), -0.12"
9,". (572), -0.11"


In [31]:
d = {"tokens":all_tokens, "attribution":attributions_sum[:len(all_tokens)].cpu()}

Add all the duplicate tokens together to get the total for each token

In [32]:
df_attrib = pd.DataFrame(d)
aggregation_functions = {'attribution': 'sum'}
df_new = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_functions)

In [33]:
highest_attrib_tokens = df_new.sort_values(by=['attribution'], ascending=False)
highest_attrib_tokens[:10]

,attribution
tokens,
Ġtraining,0.576799
-,0.231347
Ġmatch,0.222278
Ġcapt,0.199412
Ġof,0.176777
Ġwe,0.173673
Ġcaption,0.154946
Ġsuch,0.120287
Ġhuman,0.117858


In [34]:
lowest_attrib_tokens = df_new.sort_values(by=['attribution'])
lowest_attrib_tokens[:10]

,attribution
tokens,
",",-0.893572
.,-0.524259
Ġa,-0.296834
Ġthe,-0.294051
],-0.135963
Ġto,-0.090340
:,-0.087298
Ġby,-0.078813
Ġthat,-0.074638


In [35]:
print(len(all_tokens), attributions_sum[:len(all_tokens)].shape)

2050 torch.Size([2050])


In [17]:
from tqdm import tqdm
aggregate_attrib_zero = []
aggregate_attrib_ones = []
aggregation_function = {'attribution': 'sum'}

for i in tqdm(range(len(cogs402_ds))):
  text = cogs402_ds[i]['text']
  label = cogs402_ds[i]['labels']
  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)

  indices = input_ids[0].detach().tolist()
  all_tokens = tokenizer.convert_ids_to_tokens(indices)

  attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    target=label,
                                    n_steps=15,
                                    internal_batch_size = 2)
  attributions_sum = summarize_attributions(attributions)

  top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_sum)
  bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_sum)

  d = {"tokens":all_tokens, "attribution":attributions_sum[:len(all_tokens)].cpu()}  

  df_attrib = pd.DataFrame(d)
  df_attrib = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_function)

  if label == 0:
    aggregate_attrib_zero.append(df_attrib)
  else:
    aggregate_attrib_ones.append(df_attrib)

100%|██████████| 1070/1070 [5:09:11<00:00, 17.34s/it]


In [18]:
df_attrib_zero = pd.concat(aggregate_attrib_zero)
df_attrib_zero = df_attrib_zero.reset_index(level=0)
df_attrib_zero = df_attrib_zero.groupby(df_attrib_zero['tokens']).aggregate(aggregation_function)
df_attrib_zero['attribution'] = df_attrib_zero['attribution'].div(len(aggregate_attrib_zero))
highest_attrib_tokens_all = df_attrib_zero.sort_values(by=['attribution'], ascending=False)
highest_attrib_tokens_all[:10]

,attribution
tokens,
Ġthe,0.291781
",",0.284012
Ġa,0.123568
Ġthat,0.048159
),0.046686
Ġand,0.044072
Ġin,0.043083
Ġis,0.041249
1,0.032327


In [19]:
df_attrib_ones = pd.concat(aggregate_attrib_ones)
df_attrib_ones = df_attrib_ones.reset_index(level=0)
df_attrib_ones = df_attrib_ones.groupby(df_attrib_ones['tokens']).aggregate(aggregation_function)
df_attrib_ones['attribution'] = df_attrib_ones['attribution'].div(len(aggregate_attrib_ones))
highest_attrib_tokens_all2 = df_attrib_ones.sort_values(by=['attribution'], ascending=False)
highest_attrib_tokens_all2[:10]

,attribution
tokens,
.,0.493776
Ġlearning,0.102433
Ġof,0.045115
Ġtraining,0.037053
Ġneural,0.033729
ĠIn,0.028310
AI,0.023372
-,0.015785
Ġal,0.015748


In [21]:
highest_attrib_tokens_all.to_csv('longformer_emb_zeros_papers.csv')  
highest_attrib_tokens_all2.to_csv('longformer_emb_ones_papers.csv')  